# Working on videos

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
color = np.zeros((1, 1, 3))

def nothing(*args):
    pass

# Video capture object
cap = cv2.VideoCapture('./greenscreen-asteroid.mp4')
if(cap.isOpened() == False):
    print("Error opening video stream or file")

# Background replacement Image
bgImage = cv2.imread('./avengers.jpg')

# read in the first frame
ret, frame = cap.read()

# save the first frame as png image for trial
# cv2.imwrite('greenscreen.png', frame)

cv2.namedWindow("Input Video")

cv2.createTrackbar("lowH", "Input Video", 0, 179, nothing)
cv2.createTrackbar("upH", "Input Video", 179, 179, nothing)

cv2.createTrackbar("lowS", "Input Video", 0, 255, nothing)
cv2.createTrackbar("upS", "Input Video", 255, 255, nothing)

cv2.createTrackbar("lowV", "Input Video", 0, 255, nothing)
cv2.createTrackbar("upV", "Input Video", 255, 255, nothing)



# Read until video is completed
k = 0
# start from 1 instead of 0 so that last frame is not empty
frameCounter = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # to play the video in loop
    if (ret == False):
        cap.set(cv2.CAP_PROP_POS_AVI_RATIO, 0);
        continue;
    
    if ret == True and k != 27:
        
        hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lowH = cv2.getTrackbarPos("lowH", "Input Video")
        upH = cv2.getTrackbarPos("upH", "Input Video")
        
        lowS = cv2.getTrackbarPos("lowS", "Input Video")
        upS = cv2.getTrackbarPos("upS", "Input Video")
        
        lowV = cv2.getTrackbarPos("lowS", "Input Video")
        upV = cv2.getTrackbarPos("upS", "Input Video")
        
        lowGreen = np.array([lowH, lowS, lowV])
        upGreen = np.array([upH, upS, upV])
        
        mask = cv2.inRange(hsvFrame, lowGreen, upGreen)
        maskInv = cv2.bitwise_not(mask)
        
        # Gaussian blur on the mask
#         cv2.GaussianBlur()
        
        background = cv2.bitwise_and(frame, frame, mask = mask)
        foreground = cv2.bitwise_and(frame, frame, mask = maskInv)
        
        bgImageMod = cv2.bitwise_and(bgImage, bgImage, mask = mask)
        # bgImage + foreground
        outputFrame = cv2.bitwise_or(bgImageMod, foreground)
        
        cv2.imshow("Input Video", outputFrame)
        
        cv2.imshow("Background", background)
        cv2.imshow("Foreground", foreground)
        # Wait for 25 ms before moving on to the next frame
        # This will slow down the video
        k = cv2.waitKey(25)

    # Break the loop
    if k == 27:
        cv2.destroyAllWindows()
        cap.release()
        break

In [3]:
xxxxxxxxxxxxxxxxxxxxxxxxxxx

NameError: name 'xxxxxxxxxxxxxxxxxxxxxxxxxxx' is not defined

# Working on a image

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def on_mouseImage(event, x, y, flags, image):
    if event == cv2.EVENT_LBUTTONDOWN or event == cv2.EVENT_RBUTTONDOWN:
        pt = (x, y)
        
        # Getting the values of BGR at y, x
        # Hence shape is (3,)
        color = image[y, x, :]
        
        # Converting it to an image of shape (1,1,3)
        color = np.full((1,1,3), image[y,x], dtype = np.uint8)
        
        k = 0
        while (k != 27):
            cv2.imshow("Picked Color", color)
            k = cv2.waitKey(20)
        
        if k == 27:
            cv2.destroyWindow("Picked Color")

In [ ]:
# read in the image
image = cv2.imread("./greenscreen.png")
print(image.shape)

cv2.namedWindow("Image")

cv2.setMouseCallback("Image", on_mouseImage, image)

# Creating trackbar for color
# cv2.createTrackbar("")

k = 0
while (k != 27):
    cv2.imshow("Image", image)
    k = cv2.waitKey(20)
if k == 27:
    cv2.destroyWindow("Image")